In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import yfinance as yf
import time

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def scrape_wikipedia_sp500():
    """Scrape S&P 500 companies and their sectors from Wikipedia."""
    response = requests.get(WIKI_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    table = soup.find('table', {'id': 'constituents'})
    rows = table.find_all('tr')[1:]  # Skip header row
    
    companies_data = []
    for row in rows:
        cols = row.find_all('td')
        ticker = cols[0].text.strip()
        company_name = cols[1].text.strip()
        sector = cols[3].text.strip()
        companies_data.append((ticker, company_name, sector))
    
    return companies_data

def get_sp500_sectors(companies_data):
    """Return a list of unique sectors."""
    return list(set(company[2] for company in companies_data))

def get_sector_companies(sector, companies_data):
    """Return the list of companies belonging to a sector."""
    return [company[0] for company in companies_data if company[2] == sector]

def download_market_cap_data(ticker, start_date, end_date):
    """Download the stock's market capitalization at the quarter end."""
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date)
        if df.empty:
            print(f"No data found for {ticker}")
            return None

        # Calculate Market Cap = Close Price * Shares Outstanding
        shares_outstanding = stock.info.get('sharesOutstanding', None)
        if shares_outstanding is None:
            print(f"No shares outstanding data for {ticker}")
            return None
        
        df['MarketCap'] = df['Close'] * shares_outstanding
        df = df[['MarketCap']]  # Only keep MarketCap column
        
        # Resample to get the last value at the end of each quarter
        df = df.resample('QE').last()
        df['Ticker'] = ticker  # Add ticker as a column
        return df
    except Exception as e:
        print(f"Error downloading data for {ticker}: {str(e)}")
        return None

def process_sector(sector, start_date, end_date, companies_data):
    """Process each sector and calculate the market cap at quarter end."""
    companies = get_sector_companies(sector, companies_data)
    if len(companies) < 3:
        print(f"Skipping {sector} sector: Only {len(companies)} companies found.")
        return None

    print(f"Processing {sector} sector ({len(companies)} companies)...")
    print(companies)
    results = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_ticker = {executor.submit(download_market_cap_data, ticker, start_date, end_date): ticker for ticker in companies}
        for future in as_completed(future_to_ticker):
            ticker, result = future_to_ticker[future], future.result()
            if result is not None and not result.empty:
                results.append(result)
            time.sleep(2)  # To avoid overwhelming the Yahoo Finance API

    if not results:
        print(f"No valid results for {sector} sector.")
        return None

    try:
        # Combine the results into a single DataFrame
        combined_results = pd.concat(results)
        combined_results.reset_index(inplace=True)  # Reset index to get Date as a column
        combined_results.rename(columns={'index': 'Date'}, inplace=True)  # Rename the index column to Date
        return combined_results
    except ValueError as e:
        print(f"Error combining results for {sector} sector: {str(e)}")
        return None

def main():
    # Set the date range from Q1 2015 to Q2 2024 (or current date if earlier)
    start_date = pd.Timestamp('2015-01-01')
    end_date = pd.Timestamp('2024-06-30')
    end_date = min(end_date, pd.Timestamp.now())

    # Scrape Wikipedia for S&P 500 companies and sectors
    companies_data = scrape_wikipedia_sp500()

    # Get all unique sectors
    sectors = get_sp500_sectors(companies_data)

    # Create a directory for output files
    output_dir = "sector_mkt_cap_results"
    os.makedirs(output_dir, exist_ok=True)

    # Process each sector
    for sector in sectors:
        sector_results = process_sector(sector, start_date, end_date, companies_data)
        if sector_results is not None:
            # Save results to CSV
            csv_filename = os.path.join(output_dir, f"{sector}_mkt_cap_quarter_end.csv")
            sector_results.to_csv(csv_filename, index=False)  # Save without index
            print(f"Results saved to {csv_filename}")

            # Display summary
            print(f"Summary for {sector} sector:")
            print(sector_results.head())
            print("\n" + "="*50 + "\n")
        else:
            print(f"No results to save for {sector} sector.")
            print("\n" + "="*50 + "\n")

    print("All sectors processed.")

if __name__ == "__main__":
    main()
